In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [ ]:
with_mask.shape

In [ ]:
without_mask.shape

In [ ]:
X=np.r_[without_mask, with_mask]

In [ ]:
X.shape

In [ ]:
labels=np.zeros(X.shape[0])

In [ ]:
labels[1184:]=1.0

In [ ]:
names={0:'No Mask',1:'Mask'}

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(X, labels, test_size=0.20, stratify=labels)

In [ ]:
trainX.shape

In [ ]:
# Increasing dataset by augmentation
aug = ImageDataGenerator(
    rotation_range= 20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
#base model by MobileNet
baseModel = MobileNetV2(weights='imagenet', include_top=False, input_tensor= Input(shape=(224,224,3)))

#head of the model place on top of base model 
headModel = baseModel.output 
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name='flattern')(headModel)
headModel = Dense(128, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation='softmax')(headModel)

model = Model(inputs = baseModel.input, outputs = headModel)

#freezing the baseModel so that weight will not updated 
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
# Initialising value
INIT_LR = 1e-8
EPOCHS = 10
BS = 30

In [ ]:
# Compiling and training
opt = Adam(lr = INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

H = model.fit(aug.flow(trainX, trainY, batch_size=BS),
             steps_per_epoch= len(trainX) // BS,
             validation_data=(testX, testY),
             validation_steps=len(testX) // BS,
             epochs = EPOCHS,)

In [ ]:
# Predicting the model
predIdxs = model.predict(testX, batch_size = BS)

# each testing image finding largest index label probability 
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testY.argmax(axis=1), predIdxs, target_names = lb.classes_))

model.save('mask_detector.model', save_format='h5')